In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/train.csv
/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/test.csv
/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/sample_submit.csv


In [2]:
import torch

In [3]:
if torch.cuda.is_available() :
    device = 'cuda'
else :
    device = 'cpu'
    
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
import torch
import torchvision

train = pd.read_csv("../input/2021-ai-w10-p1/2021-ai-w10-p1/train.csv")
test = pd.read_csv("../input/2021-ai-w10-p1/2021-ai-w10-p1/test.csv")
submit = pd.read_csv("../input/2021-ai-w10-p1/2021-ai-w10-p1/sample_submit.csv")

In [5]:
train

,Category,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y_train = train.Category
x_train = train.drop(['Category'], axis =1)

In [7]:
x_train = np.array(x_train)
y_train = np.array(y_train)
test = np.array(test)

In [8]:
#파라미터 설정
batch_size = 100
total_epoch = 5
lr = 0.001

method = "resnet"

In [9]:
x_train.shape

(60000, 784)

In [10]:
x_train = x_train.reshape(-1,1,28,28)
x_train.shape

(60000, 1, 28, 28)

In [11]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
test = torch.FloatTensor(test)

In [12]:
from PIL import Image
import cv2
import torchvision.transforms as transforms

In [13]:
class w10_dataloader(torch.utils.data.Dataset):
    def __init__(self, x_train, y_train, transform = None ):
        self.transform = transform
        self.data = x_train
        self.label = y_train
        self.data_len = len(self.data)
        
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, index):
        
        img = np.array(self.data[index]).reshape(28,28)
        img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_LINEAR)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        img = self.transform(img)
        
        return img , self.label[index]

    
transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [14]:
class w10_test_dataloader(torch.utils.data.Dataset):
    def __init__(self, x_train, transform = None ):
        self.transform = transform
        self.data = x_train
        self.data_len = len(self.data)
        
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, index):
        
        img = np.array(self.data[index]).reshape(28,28)
        img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_LINEAR)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        img = self.transform(img)
        
        return img 

    
transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [15]:
train_data = w10_dataloader(x_train,y_train, transform = transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size , shuffle = True, num_workers = 4 )

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
test_data = w10_test_dataloader(test, transform = transform)

test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size , shuffle = False, num_workers = 4 )

In [17]:
import torchvision.models as models
model = models.resnet18(pretrained = True)
model.to(device)

model.fc.out_features= 10

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001 , momentum = 0.9)

In [19]:
from torchvision.transforms.functional import to_pil_image

In [20]:
#train my model
total_batch = len(train_loader)
print('Learning started. It takes sometime.')

for epoch in range(total_epoch):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device) # 64, 3, 224, 224 
        Y = Y.to(device)
       
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.197959706
[Epoch:    2] cost = 0.0198831744
[Epoch:    3] cost = 0.0119845206
[Epoch:    4] cost = 0.0079489667
[Epoch:    5] cost = 0.00534419529
Learning Finished!


In [21]:
prediction = []

with torch.no_grad():
    model.eval()
    
    for x in test_loader :
        x = x.to(device)
        predict = model(x)
        predict = torch.argmax(predict, dim = 1)
        
        prediction.extend(predict.detach().cpu().numpy().tolist())

In [22]:
submit['Category'] = np.array(prediction)

In [23]:
submit.to_csv("submission.csv", index = False)
submit

,Id,Category
0,0,6
1,1,1
2,2,0
3,3,0
4,4,9
...,...,...
9995,9995,3
9996,9996,7
9997,9997,2
9998,9998,8
